# Data Cleaning

In deze notebook gaan we werken met een dataset die de gebeurtenissen tijdens een reeks wedstrijden over een periode van 4 jaar.
De link naar deze dataset is: https://www.kaggle.com/secareanualin/football-events.
Met deze kan je reeds heel wat informatie vinden over wat er in deze dataset zit. 
In de dataset zitten er drie files:
* events.csv Bevat data over de gebeurtenissen van elke wedstrijd.
* ginf.csv - Bevat metadata en win-kansen van elke wedstrijd.
* dictionary.txt Bevat een tekstuele weergave van de verschillende categorieen

Hieronder staan alle imports die gebruikt worden in de loop van de notebook.

## Inladen dataset

De eerste stap is om de dataset te downloaden vanuit kaggle met de opendatasets library.
Aangezien de dataset meerdere bestanden bevat kan het zijn dat het wordt gedownload als zip (controleer dit). 
Unzip daarom eerst het bestand indien nodig.
Laad daarna de twee csv bestanden in in een dataframe.
De code voor deze stappen is:

## Exploratory Data Analysis

De read_csv functie heeft een dataframe aangemaakt en alle data van de csv ingeladen in dit object.
Pandas voorziet daarnaast een aantal eenvoudige methodes om heel wat informatie uit de dataset op te halen.
Een overzicht van alle functies die mogelijk zijn met dataframes vind je hier: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html.
We kunnen bijvoorbeeld eenvoudig het aantal rijen gaan opvragen maar ook een aantal rijen gaan bekijken. 
Dit kan bijvoorbeeld met de volgende code:

Probeer nu zelf de volgende zaken op te zoeken:
* Haal willekeurig 2 rijen uit de dataset
* Alle spelers die betrokken zijn geweest bij een event (geen duplicaten)
* Hoeveel goalen zijn er gemaakt in de geregistreerde events? Dit kan aan de hand van de is_goal kolom.
* Bestudeer de inhoud van het ginf dataset, besteed vooral aandacht aan de ontbrekende data
* Voeg aan het ginf dataframe twee kolommen toe, 1 voor het aantal events van de thuisploeg en 1 voor het aantal events van de andere ploeg.

## Data Cleaning

### Ontbrekende data

In eerste instantie zijn er de volgende mogelijkheden om om te gaan met ontbrekende data (Null or Nan values)
* Negeren
* Verwijder rijen met null waarden
* Verwijder kolommen met null waarden
* Kies een waarde om de NaN waarden in te vullen
* Gebruik duplicaten om de waarde in te vullen

Je kan gebruik maken van de volgende functies:
* isna or notna om aan te geven welke elementen ontbrekende of ingevulde data bevatten 
* dropna drop columns or rows containing missing values  (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)
* fillna fill in missing values https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna

Deze functies hebben de volgende resultaten

We zien dat de manier waarop we dropna uitvoeren een grote impact kan hebben op het resultaat.
In de eerste situatie bekomen we een lege dataset. Dit komt omdat er op elke rij minstens 1 kolom is die een Nan value bevat.

In het tweede voorbeeld, worden alle kolommen verwijderd die minstens 1 Nan value bevat. Op deze manier hebben we nog evenveel elementen maar het aantal kolommen is sterk gereduceerd. Door dit zonder controle te gebruiken kan veel nuttige informatie verwijderd worden. Merk op dat de 4 kolommen met de speler informatie verwijderd zijn. En ook de kolommen die meer informatie bevatten over het type event.

Het merendeel van de verwijderde kolommen bevatten dus nuttige informatie en willen we daarom niet verwijderen.
Om deze reden hebben we een meer manuele aanpak nodig.
De kolommen met informatie over het event (bvb shot_place) zijn nodig en mogen dus blijven staan. 
Nan geeft dan enkel weer dat de waarde niet van toepassing is.
De 4 kolommen met speler informatie kan echter gereduceerd worden tot twee kolommen (1 voor elk team) waarbij player2 niet altijd ingevuld moet zijn.
Dit kan bewezen worden als volgt: 

Met bovenstaande resultaten zien we dat in bijna alle gevallen player ofwel player_in ingevuld is maar nooit allebei.
Dit houdt in dat we deze twee kolommen eenvoudig kunnen samenvoegen. 
Hetzelfde geldt voor de kolommen player2 en player_out.
Nog een belangrijke opmerking is dat er een aantal rijen zijn waar er geen informatie over spelers aanwezig is.
Deze rijen kunnen verwijderd worden.

Overbodige kolommen:
* text
* id_event (samenstelling van id_odsp en sort_order)

Dit doen we hier:

Oefening: Zijn er na het mergen van deze twee kolommen nog gevallen waar player null is maar player2 niet? In het geval dat er nog zo rijen zijn, verplaats de waarde van de player2 kolom naar de player kolom (enkel voor deze rijen). Nadat dit gedaan is hebben we zoveel mogelijk gedaan en kunnen we de overige rijen waarvoor de player feature null is laten vallen. 

### One hot encoding

In deze dataset wordt er vaak een enum gebruikt om een categorie aan te duiden. 
Dit is een compacte manier om een enum in een dataset op te slaan maar dit geeft vaak problemen bij het opbouwen van modellen.
Dit gaan we in meer detail bekijken bij classificatie/clustering.

Om dit op te lossen kunnen gebruik maken van one-hot encoding.
In pandas kan dat door gebruik te maken van de functie pd.get_dummies()

### Data Format Correcting

Buiten het verwijderen of toevoegen van kolommen kan het ook belangrijk zijn om de waarden van individuele elementen te corrigeren/aan te passen. 
Dit kan een heel aantal zaken omvatten zoals het corrigeren van typo's, omzetten van datatypes, etc.
Laten we eerst eens de One-Hot Encoded values omzetten van 0/1 naar true/false.

Daarnaast moeten er ook vaak woorden in tekstuele velden gecorrigeerd of vertaald worden. 
Om hiervan een voorbeeld te geven ga ik in de player en player2 kolommen de naam van "Romelu Lukaku" hernoemen naar "Big Rom".
De volgende code voert dit uit door gebruik te maken van de [replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html) functie:

Een andere grote bron van data formaten dat aangepast moet worden komt voort uit kolommen die datums bevatten. Deze worden standaard ingelezen als een tekst veldje dus moet je deze zelf gaan omzetten. Dit kan je zien in de ginf dataset waar de "date" kolom van het type object is. Dit is in numpy de indicatie dat het een tekstveld is.

In de dataframe zien we dat de datums erin staan als "%Y-%m-%d" (jaar als 4 cijfers, rest als twee). 
Voor de notatie van andere dataformaten vind je [hier](http://strftime.org/) meer informatie.
Met behulp van de [to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) functie kan je deze kolom omzetten naar datums.
Zoek na deze omzetting eens het datatype op van deze kolom.
Je ziet dat het nu geen object meer is maar een correcte datum.

Merk op dat deze manier van aanpakken enkel werkt wanneer de hele tabel hetzelfde dataformaat heeft.
Indien er meerdere dataformaten gebruikt zijn (bijvoorbeeld door manuele input), dan kan je het nog steeds doen maar dan moet panda een inschatting maken van het gewenste formaat. Dit kan door het argument "infer_datetime_format=True".
Hoewel dit mogelijk is zijn er twee redenen waarom we dit niet altijd gebruiken, namelijk
* Het duurt veel langer om de tabel om te zetten.
* Het gaat niet altijd het correcte resultaat geven bij speciale formaten

Eenmaal de kolom van het type datetime is kan je heel gemakkelijk deze kolommen vergelijken en er zaken gaan uithalen zoals de maand waarin de wedstrijd plaatsvond. 

### Data shuffling

Een mogelijke manier om de privacy van je klanten te beschermen die we aangehaald hebben is data shuffling.
Bij deze methode is het de bedoeling dat je de namen in een dataset op een willekeurige manier door elkaar haalt.
Dit kan gedaan worden door gebruik te maken van de permutation methode van numpy  (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) op de volgende manier:

## Oefeningen

Om meer vertrouwd te worden met het werken met pandas kan je de volgende zaken zelf proberen.

Bewerkingen op de events.csv dataset:
* Controleer de kolommen "id_event", "id_odsp" en "sort_order". Is er 1 van deze drie overbodig? Verwijder de kolom indien dit zo is.
* Hernoem "Hamburg SV" naar "Hamburg FC". Let op, doe dit voor alle kolommen waar het kan staan.

Bewerkingen op de ginf.csv dataset:
* Maak One Hot Encoding voor de kolom "Country"
* Voeg een kolom toe die drie waarden kan hebben: Home team wins, Home team loses, draw
* Maak nu ook een One Hot Encoding voor deze nieuwe kolom
* Drop de kolom met de urls voor de ploegen
* Bewaar je aangepaste dataset in "wedstrijden.csv" (Gebruik de functie to_save: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)

## Oefening 2

Gebruik de dataset die je kan vinden op [deze link](https://gist.githubusercontent.com/deanjamesss/f8f5781fb682fb728e9674dda9813982/raw/801df9e7b9af894af1f69ba1a40af909160891aa/employee_data.csv).
Deze dataset bevat informatie over de personeelslijst van een bedrijf.
Voer de volgende stappen/oefeningen/bewerkingen uit:
* Bekijk de datatypes van de verschillende kolommen. Is alles zoals je verwacht?
* Bereken het aantal nulwaarden per kolom.
* Verwijder de kolom met de gegevens over het belastingsnummer. Verliezen we hiermee data?
* Verwijder alle rijen die meer dan 2 ontbrekende gegevens bevatten. Hoeveel rijen blijven er over?
* Geef een dataframe met de lijsten verwijderd in de vorige stap.
* Vul in de kolommen voor het geslacht en employment_status ontbrekende waarden in met een U van unknown.
* Om de geslacht kolom leesbaarder te maken, vervang M door male, F door female en U door unknown.
* Voeg een one-hot encoding voor de kolom van de employment_status toe. Verwijder ook de originele kolom.
* Bewaar het bewerkte csv in een bestand